In [ ]:
%sql
select * from prod.uc_01_acl.customers

In [ ]:
%sql 
CREATE OR REPLACE FUNCTION main.jpg.simple_filter(country STRING)
RETURN 
  is_account_group_member('tuanis_admins')
  OR
  country IN ('Brazil', 'United States')
;

In [ ]:
%sql
ALTER TABLE main.jpg.customers 
SET ROW FILTER main.jpg.simple_filter ON (country);


In [ ]:
%sql
CREATE TABLE main.jpg.map_rls_country (
  group STRING,
  countries ARRAY<STRING>
);

In [ ]:
%sql
INSERT INTO main.jpg.map_rls_country (group, countries) VALUES
  ('northamer', ARRAY('USA', 'CANADA', 'MEXICO')),
  ('europe', ARRAY('FR', 'SPAIN'));

In [ ]:
%sql 
CREATE OR REPLACE FUNCTION main.jpg.country_row_filter(country STRING)
RETURN 
  is_account_group_member('tuanis_admins')
  OR
  EXISTS (
    SELECT 1
    FROM prod.default.map_rls_country m
    WHERE 
    array_contains(m.countries, country)
    AND is_account_group_member(m.group)
  )
;


In [ ]:
%sql
CREATE OR REPLACE FUNCTION main.jpg.mask_email(email STRING)
RETURN
  IF(
    is_account_group_member('tuanis_admins'),
    email,
    CONCAT('***@', SPLIT(email, '@')[1])
  )
;
